In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
import numpy as np
import csv

In [ ]:
begin, end = 6, 61 # (begin is inclusive, end is exclusive)
count_samples = {
    "active": 16,
    "meditate": 15,
    "neutral": 15
}

Define `Sample` class to store data

In [ ]:
class Sample:
    def __init__(self):
        self.data = {
            'RawEEG': [],
            'Alpha': [],
            'Low Beta': [],
            'High Beta': [],
            'Gamma': [],
            'Theta': [],
            'Delta': [],
            'Meditation': [],
            'Attention': []
        }

    def recordDataPoint(self, RawEEG, Attention, Meditation, Alpha, Delta, Theta, LowBeta, HighBeta, Gamma):
        self.data['RawEEG'].append(float(RawEEG))
        self.data['Attention'].append(float(Attention))
        self.data['Meditation'].append(float(Meditation))
        self.data['Alpha'].append(float(Alpha))
        self.data['Delta'].append(float(Delta))
        self.data['Theta'].append(float(Theta))
        self.data['Low Beta'].append(float(LowBeta))
        self.data['High Beta'].append(float(HighBeta))
        self.data['Gamma'].append(float(Gamma))

    '''
    Record a line of data from the CSV output, which takes form RawEEG, Alpha, Delta, Gamma, Low Beta, High Beta, Theta, Attention, Meditation

    '''
    def recordDataLine(self, line):
        self.recordDataPoint(line[0], line[7], line[8], line[1], line[2], line[6], line[4], line[5], line[3])
    
    def getEEG(self):
        return self.data['RawEEG']
    
    def getAttention(self):
        return self.data["Attention"]
    
    def getMeditation(self):
        return self.data["Meditation"]
    
    def getAlpha(self):
        return self.data["Alpha"]
    
    def getDelta(self):
        return self.data["Delta"]
    
    def getTheta(self):
        return self.data["Theta"]
    
    def getLowBeta(self):
        return self.data["Low Beta"]
    
    def getHighBeta(self):
        return self.data["High Beta"]
    
    def getGamma(self):
        return self.data["Gamma"]

    def get(self, key):
        return self.data[key]

    '''
    Scales the data by standard deviation of the EEG data
    '''
    def scale(self):
        eeg_std_dev = np.std(self.data['RawEEG'])
        self.data['Alpha'] = [x / eeg_std_dev for x in self.data['Alpha']]
        self.data['Delta'] = [x / eeg_std_dev for x in self.data['Delta']]
        self.data['Theta'] = [x / eeg_std_dev for x in self.data['Theta']]
        self.data['Low Beta'] = [x / eeg_std_dev for x in self.data['Low Beta']]
        self.data['High Beta'] = [x / eeg_std_dev for x in self.data['High Beta']]
        self.data['Gamma'] = [x / eeg_std_dev for x in self.data['Gamma']]

Transcribe from file to samples stored in the `data` and `dataLabels` lists

In [ ]:
data = []
dataLabels = []

def transcribeFileToSample(sampleN: int, state: str):
    sample_data = Sample()

    with open("raw_data/" + state + str(sampleN) + ".csv") as f:
        reader = csv.reader(f)

        header = next(reader)
        
        for row in reader:
            sample_data.recordDataLine(row)

        for key in sample_data.data:
            sample_data.data[key] = sample_data.data[key][begin:end]

        sample_data.scale()

        data.append(sample_data)
        dataLabels.append(state)

for state in count_samples:
    for i in range(count_samples[state]):
        transcribeFileToSample(i + 1, state)

Extract features from raw data and store in `dataExtracted`

In [ ]:
dataExtracted = []

for point in data:
    extractedPoint = []

    extractedPoint.append(np.mean(point.getAlpha()))
    extractedPoint.append(np.mean(point.getLowBeta()))
    extractedPoint.append(np.std(point.getAlpha()))
    extractedPoint.append(np.mean(point.getTheta()))
    extractedPoint.append(np.std(point.getGamma()))
    extractedPoint.append(np.std(point.getDelta()))

    dataExtracted.append(extractedPoint)


Cross validate pipeline

In [ ]:
# cvclf = make_pipeline(StandardScaler(), LinearDiscriminantAnalysis(n_components=1))
# scores = cross_val_score(cvclf, dataExtracted, dataLabels, cv=5)
# print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Train/test split and train different classifiers - average over many trials

In [ ]:
total = 0
n = 10000
errorLabels = {"active": {"neutral": 0, "meditate": 0}, 
            "neutral": {"active": 0, "meditate": 0}, 
            "meditate": {"active": 0, "neutral": 0}}
            # first key = actual, second key = prediction

for _ in range(n):
    train, test, trainLabels, testLabels = train_test_split(dataExtracted, dataLabels, test_size=0.20)

    clf = make_pipeline(StandardScaler(), LinearDiscriminantAnalysis(n_components=1))
    clf.fit(train, trainLabels)

    predictions = clf.predict(test)
    for i in range(len(predictions)):
        if predictions[i] != testLabels[i]:
            errorLabels[testLabels[i]][predictions[i]] += 1 
            
    total += clf.score(test, testLabels) 

print(total/n)
print(errorLabels)

In [ ]:
cv_begin = 2
cv_end = 31

cv_data = []
cv_dataLabels = []

cv_samples = {
    "active": 5,
    "neutral": 5,
    "meditate": 5
}

def cv_transcribeFileToSample(sampleN: int, state: str):
    sample_data = Sample()

    with open("raw_cv_data/" + state + str(sampleN) + ".csv") as f:
        reader = csv.reader(f)

        header = next(reader)
        
        for row in reader:
            sample_data.recordDataLine(row)

        for key in sample_data.data:
            sample_data.data[key] = sample_data.data[key][cv_begin:cv_end]

        sample_data.scale()

        cv_data.append(sample_data)
        cv_dataLabels.append(state)

for state in cv_samples:
    for i in range(cv_samples[state]):
        cv_transcribeFileToSample(i + 1, state)

In [ ]:
cv_extracted = []

for point in cv_data:
    extractedPoint = []

    extractedPoint.append(np.mean(point.getAlpha()))
    extractedPoint.append(np.mean(point.getLowBeta()))
    extractedPoint.append(np.std(point.getAlpha()))
    extractedPoint.append(np.mean(point.getTheta()))
    extractedPoint.append(np.std(point.getGamma()))
    extractedPoint.append(np.std(point.getDelta()))

    cv_extracted.append(extractedPoint)


In [15]:
predictions = clf.predict(cv_extracted)

print("Predictions: ", predictions)
print("Real Labels: ", cv_dataLabels)
print("Correct " + str(100*clf.score(cv_extracted, cv_dataLabels)) + "% of the time!")

Predictions:  ['neutral' 'neutral' 'meditate' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'active']
Real Labels:  ['active', 'active', 'active', 'active', 'active', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'meditate', 'meditate', 'meditate', 'meditate', 'meditate']
Correct 33.33333333333333% of the time!
